# Quantum Superposition: Beyond Classical Probability

## Introduction

**What is superposition?**  
Superposition is the fundamental principle that distinguishes quantum computing from classical computing. While a classical bit can only be 0 or 1, a quantum bit (qubit) can exist in a *superposition* of both states simultaneously.

**Why it matters for quantum computing**  
Superposition allows quantum computers to explore multiple computational paths at once, forming the foundation of quantum parallelism. Every quantum algorithm leverages superposition to achieve computational advantages.

**Classical analogy vs quantum reality**  
At first glance, superposition might seem similar to classical probability (like a coin flip). However, there's a profound difference:
- **Classical probability**: The coin is either heads or tails; we just don't know which until we look
- **Quantum superposition**: The qubit genuinely exists in both states at once until measured

**What we'll demonstrate**  
In this notebook, we'll:
1. Simulate classical probability with Monte Carlo methods
2. Create quantum superposition using the Hadamard gate
3. Visualize the difference through beautiful plots
4. Explore quantum states on the Bloch sphere
5. (Optional) Run on real quantum hardware

**Key equation**  
A qubit in superposition is described by:

$$|\psi\rangle = \alpha|0\rangle + \beta|1\rangle$$

where $\alpha$ and $\beta$ are complex amplitudes satisfying $|\alpha|^2 + |\beta|^2 = 1$, and $|\alpha|^2$ and $|\beta|^2$ are the probabilities of measuring $|0\rangle$ or $|1\rangle$.

## Setup: Imports and Configuration

Let's import our quantum computing tools and set up beautiful visualizations.

In [ ]:
# Quantum computing framework
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_bloch_multivector

# Numerical and visualization libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Custom utilities for beautiful plots
import sys
sys.path.append('..')
from utils.plotting import (
    configure_beautiful_plots,
    plot_histogram_comparison,
    plot_bloch_sphere,
    plot_circuit,
    COLORS
)

# Configure beautiful plotting style
configure_beautiful_plots()

# Set random seed for reproducibility
np.random.seed(42)

print("✅ All imports successful")
print(f"✅ Qiskit ready for quantum exploration")

## Classical Probability: Monte Carlo Coin Flips

**What we'll do**  
Simulate a fair coin flip 1000 times using classical random number generation.

**Why**  
This establishes our classical baseline. A fair coin gives heads or tails with 50% probability each. The coin is always in one definite state; randomness comes from our lack of knowledge.

**How**  
Use NumPy's random number generator to simulate coin flips, then count and visualize the outcomes.

**Expected result**  
Approximately 50/50 distribution between heads (0) and tails (1), with small statistical fluctuations.

In [ ]:
# Monte Carlo simulation of fair coin flips
n_trials = 1000

# Generate random coin flips (0 = heads, 1 = tails)
classical_flips = np.random.randint(0, 2, size=n_trials)

# Count outcomes
classical_counts = {
    '0': np.sum(classical_flips == 0),
    '1': np.sum(classical_flips == 1)
}

print(f"Classical coin flips ({n_trials} trials):")
print(f"  Heads (0): {classical_counts['0']} ({classical_counts['0']/n_trials*100:.1f}%)")
print(f"  Tails (1): {classical_counts['1']} ({classical_counts['1']/n_trials*100:.1f}%)")

# Beautiful histogram
fig, ax = plt.subplots(figsize=(8, 6), dpi=150)
bars = ax.bar(['0 (Heads)', '1 (Tails)'], 
              [classical_counts['0'], classical_counts['1']],
              color=COLORS['classical'], alpha=0.8, 
              edgecolor='black', linewidth=2)

ax.set_title('Classical Probability: Fair Coin Flips', 
             fontsize=16, fontweight='bold', pad=15)
ax.set_xlabel('Outcome', fontsize=12)
ax.set_ylabel('Counts', fontsize=12)
ax.grid(axis='y', alpha=0.3)

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height)}',
            ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

## Classical Results: Understanding the Distribution

The histogram shows approximately equal counts for heads and tails, as expected from a fair coin. 

**Key insight**: In classical probability, each coin flip produces a definite outcome (either 0 or 1). The randomness reflects our ignorance about the initial conditions, not a fundamental property of the coin itself.

Now let's see how quantum mechanics differs...

## Quantum Superposition: The Hadamard Gate

**What we'll do**  
Create a quantum circuit that puts a qubit into superposition using the Hadamard (H) gate.

**Why**  
The Hadamard gate is the fundamental tool for creating superposition. It transforms the definite state $|0\rangle$ into an equal superposition of $|0\rangle$ and $|1\rangle$.

**How**  
1. Initialize a qubit in state $|0\rangle$ (the default)
2. Apply the Hadamard gate
3. Measure the qubit
4. Repeat 1000 times to gather statistics

**Expected result**  
50/50 distribution between 0 and 1 (same as classical!), but the underlying physics is completely different.

**Circuit diagram**  
```
     ┌───┐┌─┐
q_0: ┤ H ├┤M├
     └───┘└╥┘
c: 1/══════╩═
           0 
```

**Mathematical transformation**  
The Hadamard gate transforms:

$$H|0\rangle = \frac{1}{\sqrt{2}}(|0\rangle + |1\rangle)$$

This creates a **genuine quantum superposition** where the qubit simultaneously exists in both states.

In [ ]:
# Create quantum circuit with 1 qubit and 1 classical bit
qc = QuantumCircuit(1, 1)

# Apply Hadamard gate to create superposition
qc.h(0)

# Measure the qubit
qc.measure(0, 0)

# Draw the circuit
print("Quantum Circuit:")
display(qc.draw('mpl', style='iqp'))

# Run on quantum simulator
simulator = AerSimulator()
job = simulator.run(qc, shots=1000)
result = job.result()
quantum_counts = result.get_counts()

print(f"\nQuantum superposition measurements (1000 shots):")
for state, count in sorted(quantum_counts.items()):
    print(f"  |{state}⟩: {count} ({count/1000*100:.1f}%)")

In [ ]:
# Beautiful histogram of quantum results
fig, ax = plt.subplots(figsize=(8, 6), dpi=150)

states = sorted(quantum_counts.keys())
counts = [quantum_counts[s] for s in states]
labels = [f'|{s}⟩' for s in states]

bars = ax.bar(labels, counts,
              color=COLORS['quantum'], alpha=0.8,
              edgecolor='black', linewidth=2)

ax.set_title('Quantum Superposition: Hadamard Gate Results',
             fontsize=16, fontweight='bold', pad=15)
ax.set_xlabel('Quantum State', fontsize=12)
ax.set_ylabel('Counts', fontsize=12)
ax.grid(axis='y', alpha=0.3)

# Add value labels
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height)}',
            ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

## Quantum Results: The Measurement Paradox

The measurement statistics look identical to the classical coin flip. So what's different?

**The crucial difference**:
- **Classical**: Each coin flip was *always* either heads or tails; we just didn't know which
- **Quantum**: Before measurement, the qubit genuinely existed in *both* states simultaneously

**Key insights**:
1. The quantum state before measurement is $\frac{1}{\sqrt{2}}(|0\rangle + |1\rangle)$
2. This is a **genuine superposition**, not just uncertainty
3. Measurement **collapses** the superposition into one definite outcome
4. The probabilities ($|\alpha|^2$ and $|\beta|^2$) emerge from the amplitudes

This difference becomes crucial for quantum algorithms, where we manipulate amplitudes through interference before measurement.

## Bloch Sphere: Visualizing Quantum States

**What is the Bloch sphere?**  
The Bloch sphere is a geometric representation of a single-qubit quantum state. It's a powerful visualization tool that makes quantum mechanics more intuitive.

**Why visualize on the Bloch sphere?**  
- Shows quantum states as points on a sphere
- North pole = $|0\rangle$, South pole = $|1\rangle$
- Superpositions live on the surface
- Makes quantum operations visible as rotations

**What we'll see**:
- Initial state $|0\rangle$ at the north pole
- After Hadamard gate: state moves to the equator (equal superposition)

**Expected**:  
The Hadamard gate rotates $|0\rangle$ from the north pole to a point on the equator, representing the superposition state $\frac{1}{\sqrt{2}}(|0\rangle + |1\rangle)$.

In [ ]:
# Create circuit WITHOUT measurement to see the state
qc_statevector = QuantumCircuit(1)

# Initial state |0⟩
initial_state = Statevector(qc_statevector)

# Apply Hadamard gate
qc_statevector.h(0)
superposition_state = Statevector(qc_statevector)

print("Initial state |0⟩:")
print(f"  Amplitudes: {initial_state.data}")
print(f"  Probabilities: |α|² = {np.abs(initial_state.data[0])**2:.3f}, |β|² = {np.abs(initial_state.data[1])**2:.3f}")

print("\nAfter Hadamard gate:")
print(f"  Amplitudes: {superposition_state.data}")
print(f"  Probabilities: |α|² = {np.abs(superposition_state.data[0])**2:.3f}, |β|² = {np.abs(superposition_state.data[1])**2:.3f}")

In [ ]:
# Visualize both states on Bloch sphere
from qiskit.visualization import plot_bloch_multivector

# Plot initial state
print("Initial State |0⟩:")
display(plot_bloch_multivector(initial_state))

# Plot superposition state
print("\nSuperposition State H|0⟩:")
display(plot_bloch_multivector(superposition_state))

print("\n📍 State |0⟩ appears at the north pole (z-axis)")
print("📍 Superposition state appears on the equator (x-axis)")
print("\nThe Hadamard gate rotates the state from north pole to equator.")

## Side-by-Side Comparison: Classical vs Quantum

**Why compare?**  
While the measurement statistics look similar, the underlying mechanisms are fundamentally different.

**Key differences to observe**:
1. **Classical**: Definite states throughout, randomness from ignorance
2. **Quantum**: Genuine superposition before measurement, collapse upon observation
3. **Statistics**: Both show ~50/50 distribution (as expected)
4. **Physics**: Completely different explanations for the same statistics

This distinction becomes critical when we explore interference and entanglement in later notebooks.

In [ ]:
# Beautiful side-by-side comparison
fig = plot_histogram_comparison(
    classical_counts,
    quantum_counts,
    title1='Classical Probability',
    title2='Quantum Superposition',
    overall_title='Classical vs Quantum: Same Statistics, Different Physics'
)

plt.show()

print("\n" + "="*70)
print("CLASSICAL vs QUANTUM")
print("="*70)
print("\nClassical (Random bit):")
print("  • Each trial: bit is EITHER 0 OR 1 (definite state)")
print("  • Randomness: from our ignorance of initial conditions")
print("  • Statistics: 50/50 due to fair random process")
print("\nQuantum (Superposition):")
print("  • Before measurement: qubit is BOTH 0 AND 1 simultaneously")
print("  • Randomness: fundamental property of quantum mechanics")
print("  • Measurement: collapses superposition to definite state")
print("  • Statistics: 50/50 from equal superposition amplitudes")
print("\nThe difference enables quantum interference and entanglement.")
print("="*70)

## Deep Dive: Classical vs Quantum Paradigms

The identical 50/50 statistics mask a profound difference in how nature works at the quantum level. Let's explore this distinction in detail.

### Classical Paradigm: Hidden Information

In the classical world, randomness reflects **incomplete information**:

1. **Definite Reality**: The coin is always in a definite state (heads OR tails)
2. **Deterministic**: Given perfect knowledge of initial conditions (position, velocity, air resistance), we could predict the outcome
3. **Observer Independence**: The coin's state exists independently of whether we observe it
4. **Local**: The coin's state is stored locally in its physical properties

**Mathematical representation**:
- State: $s \in \{0, 1\}$ (definite value)
- Probability: $P(s)$ represents our ignorance, not the coin itself
- Measurement: Reveals pre-existing information

### Quantum Paradigm: Fundamental Uncertainty

In the quantum world, superposition is a **genuine physical state**:

1. **Superposition Reality**: The qubit literally exists in both states simultaneously
2. **Fundamental Randomness**: Even with perfect knowledge of the quantum state, outcomes are inherently probabilistic
3. **Observer Dependence**: Measurement fundamentally alters the system (collapse)
4. **Non-local**: Quantum information is stored in complex amplitudes that can interfere

**Mathematical representation**:
- State: $|\psi\rangle = \alpha|0\rangle + \beta|1\rangle$ (superposition)
- Amplitudes: $\alpha, \beta \in \mathbb{C}$ are complex numbers (can be negative or imaginary!)
- Probabilities: $P(0) = |\alpha|^2$, $P(1) = |\beta|^2$
- Measurement: **Creates** the definite outcome, doesn't just reveal it

### Key Experimental Difference: Interference

The crucial difference becomes apparent when we can make amplitudes interfere:

**Classical**: 
- Probabilities always add: $P_{total} = P_1 + P_2$
- No cancellation possible (probabilities are always positive)

**Quantum**: 
- Amplitudes interfere before squaring: $|\alpha_1 + \alpha_2|^2 \neq |\alpha_1|^2 + |\alpha_2|^2$
- **Negative amplitudes can cancel**: Destructive interference
- **Positive amplitudes enhance**: Constructive interference

This is why we can build quantum algorithms that explore many paths in superposition, then use interference to amplify correct answers and cancel wrong ones.

### The Bell Test Connection

While we haven't covered entanglement yet, it's worth noting that Bell's theorem (1964) and subsequent experiments have proven that quantum mechanics cannot be explained by any theory of local hidden variables. The quantum world is fundamentally different from classical intuition.

### Practical Implications for Quantum Computing

1. **Parallelism**: A quantum computer with $n$ qubits can be in a superposition of all $2^n$ states simultaneously
2. **Interference**: Quantum algorithms use interference to amplify correct solutions
3. **Fragility**: Superposition is delicate—any interaction with the environment causes decoherence (collapse to classical)
4. **Measurement Cost**: Measurement destroys superposition, so we must design algorithms carefully

**Bottom line**: The 50/50 distribution we see is the same, but the journey to get there is fundamentally different. Classical: pick one definite path. Quantum: explore all paths simultaneously, then collapse to one outcome upon measurement.

## Hardware Execution: Real Quantum Computer (Optional)

**What we'll do**  
Run our superposition circuit on real quantum hardware (Compute Canada Monarch).

**Why**  
Real quantum computers have noise, decoherence, and imperfect gates. Comparing simulator vs hardware shows the challenges of real quantum computing.

**How**  
Submit our circuit to the Monarch backend and retrieve results.

**Expected**  
Results will be similar to the simulator but with small deviations due to:
- Gate errors (imperfect Hadamard gate)
- Readout errors (measurement mistakes)
- Decoherence (quantum state degradation)

**Note**: This requires Compute Canada credentials. If not configured, the cell will show a placeholder message.

In [ ]:
# Monarch hardware execution (placeholder)
from utils.monarch_config import MonarchConfig, print_hardware_info

# Print hardware info (placeholder)
print_hardware_info()

print("\n" + "⚠"*35)
print("HARDWARE EXECUTION PLACEHOLDER")
print("⚠"*35)
print("\nTo run on Compute Canada Monarch quantum computer:")
print("1. Configure credentials in utils/monarch_config.py")
print("2. Initialize MonarchConfig and connect to backend")
print("3. Submit circuit with: config.submit_job(qc, shots=1000)")
print("4. Compare results with simulator")
print("\nExpected hardware results:")
print("  • Similar distribution to simulator (~50/50)")
print("  • Small deviations due to gate errors (~1-5%)")
print("  • Possible readout errors (~1-3%)")
print("  • Overall fidelity: typically 0.95-0.98")
print("\nThe difference shows real-world quantum computing challenges.")
print("⚠"*35)

## Summary: Superposition Fundamentals

**What we learned**:

1. **Classical probability** vs **quantum superposition**
   - Classical: definite states, apparent randomness
   - Quantum: genuine superposition, fundamental randomness

2. **The Hadamard gate** creates equal superposition
   - Transforms $|0\rangle \rightarrow \frac{1}{\sqrt{2}}(|0\rangle + |1\rangle)$
   - Fundamental building block for quantum algorithms

3. **Measurement collapses superposition**
   - Before: qubit in both states
   - After: definite outcome (0 or 1)
   - Probabilities from amplitude squared: $P(0) = |\alpha|^2$

4. **Bloch sphere visualization**
   - Geometric representation of quantum states
   - Operations as rotations on the sphere

**Next steps**:

In the next notebook, we'll explore **quantum interference** - how superposition amplitudes can interfere constructively or destructively to solve computational problems. This is where quantum computing's true power begins to emerge.

---

*"If you think you understand quantum mechanics, you don't understand quantum mechanics."*  
— Richard Feynman

But we're making progress. 🚀✨